In [1]:
import keras
keras.__version__

ModuleNotFoundError: No module named 'keras'

# Первый взгляд на нейронную сеть

Здесь содержатся примеры кода, приведенные в разделе 1 главы 2 [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Обратите внимание, что оригинальный текст содержит гораздо больше контента, в частности, дополнительные пояснения и рисунки: в этом блоке статей вы найдете только исходный код и соответствующие комментарии.

----

Давайте рассмотрим наш первый конкретный пример нейронной сети, которая использует библиотеку Python Keras, чтобы научиться распозновать
рукописные цифры. Если у вас еще нет опыта работы с Keras или подобными библиотеками, вы не сразу поймете содержание данного примера. Возможно, вы еще даже не установили Keras. Не волнуйтесь, это нормально. В следующей главе мы
рассмотрим каждый элемент в нашем примере и объясним их подробно. Так что не волнуйтесь, если некоторые шаги кажутся вам непонятными!
Должны же мы откуда то начать.

Проблема, которую мы пытаемся решить, состоит в том, чтобы классифицировать серые цифры, написанные от руки (28 на 28 пикселей), (от 0 до 9). Набор данных, который мы будем использовать, - это набор данных MNIST, классический набор данных в сообществе машинного обучения, который используется почти столько же времени, сколько сама коцепция МО и очень интенсивно изучается. Это набор из 60000 тренировочных образцов, плюс 10000 тестовых
изображения, собранные Национальным Институтом Стандартов и Технологий (NIST в MNIST) в 1980-х годах. Вы можете рассматривать «решение» MNIST как «Hello World» в мире Deep Learning - это то, что вы делаете, чтобы убедиться, что ваши алгоритмы работают должным образом. Как вы углубитесь в МО, вы увидите, что MNIST снова и снова появляется в научных статьях, сообщениях в блогах и так далее.

Набор данных MNIST предоставляется в Keras в виде набора из четырех массивов Numpy:

In [2]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

`train_images` и `train_labels` образуют тренировочный сет, данные, на которых будет обучаться сеть. Затем модель будет проверена с помощью "тестового сета", `test_images` и `test_labels`. Наши изображения кодируются как массивы Numpy, а метки представляют собой просто массив цифр, начиная с
от 0 до 9. Между изображениями и надписями существует однозначное соответствие.

Давайте посмотрим на данные обучения:

In [3]:
train_images.shape

(60000, 28, 28)

In [4]:
len(train_labels)

60000

In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

Взглянем на тестовые данные:

In [6]:
test_images.shape

(10000, 28, 28)

In [7]:
len(test_labels)

10000

In [8]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)


Наш рабочий процесс будет следующим: сначала мы представим нашу нейронную сеть с данными обучения, `train_images` и `train_labels`. Затем сеть научится связывать изображения и метки. Наконец, мы попросим сеть подготовить прогнозы для `test_images` и проверим, соответствуют ли эти прогнозы меткам из `test_labels`.

Что ж, давайте создадим нашу сеть - опять же, не забывайте, что вы пока можете понимать не все, описанное в этом примере. Это нормально.

In [9]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))


Основным строительным блоком нейронных сетей является «слой», модуль обработки данных, который вы можете представить как «фильтр» для данных. Данные приходят и выходят из них в более полезной форме. Именно слои извлекают представления из данных полученных ими - они более значимы для рассматриваемой проблемы. Большая часть машинного обучения на самом деле состоит из объединения простых слоев, которые  реализуют форму прогрессивной «дистилляции данных». Модель глубокого обучения подобна решетке для обработки данных, состоящей из последовательности все более совершенных фильтров данных - «слоев».

Наша сеть состоит из последовательности двух `Dense` слоев, которые являются плотно связанными (также называемыми «полностью связанными») нейронными слоями.
Второй (и последний) уровень - это 10-слойный слой «softmax», он вернет массив из 10 вероятностных оценок. Каждая
оценка - это вероятность того, что текущее цифровое изображение принадлежит одному из наших 10-значных классов.

Чтобы подготовить нашу сеть к обучению, нам нужно добавить еще три момента в рамках этапа «компиляции»:

* Функция потери: то, как сеть сможет измерить, насколько хорошо она выполняет свои учебные данные, и, таким образом, насколько правильно она выполнит задачи
* Оптимизатор: это механизм, посредством которого сеть обновляет себя на основе данных, которые она принимает, и своей функции потери.
* Метрики для мониторинга во время обучения и тестирования. Здесь мы будем заботиться только о точности (доля изображений, которые были правильно классифицированы).

Назначение функции потери и оптимизатора будет описана в следующих двух главах.

In [10]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

Перед обучением мы будем предварительно обрабатывать наши данные, преобразовывая их в форму, ожидаемую сетью, и масштабируя ее так, чтобы все значения находились в интервале [0, 1]. Ранее наши обучающие изображения, например, хранились в массиве фигур (60000, 28, 28) типа uint8 со значениями в интервале [0, 255]. Мы преобразуем его в массив типа float32 (60000, 28 * 28) со значениями от 0 до 1.

In [11]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

Нам также необходимо кодировать метки, шаг, который мы объясним в главе 3:

In [12]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Теперь мы готовы обучить нашу сеть, что в Keras осуществляется посредством вызова метода `fit` сети:

In [13]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 2s - loss: 0.2577 - acc: 0.9245     
Epoch 2/5
60000/60000 [==============================] - 1s - loss: 0.1042 - acc: 0.9690     
Epoch 3/5
60000/60000 [==============================] - 1s - loss: 0.0687 - acc: 0.9793     
Epoch 4/5
60000/60000 [==============================] - 1s - loss: 0.0508 - acc: 0.9848     
Epoch 5/5
60000/60000 [==============================] - 1s - loss: 0.0382 - acc: 0.9890     


Во время обучения отображаются две величины: «потеря» сетью части данных обучения и точность сети по сравнению с данными обучения.

Мы быстро достигаем точности 0,989 (т. е. 98,9%) по данным обучения. Теперь давайте проверим, также ли хорошо модель работает на тестовом наборе:

In [14]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

 9536/10000 [===========================>..] - ETA: 0s

In [15]:
print('test_acc:', test_acc)

test_acc: 0.9777



Точность наших тестовых наборов составляет 97,8% - это немного ниже, чем точность тренировочных наборов.
Этот разрыв между точностью обучения и точностью теста является примером "переоснащения",
это факт, что модели машинного обучения имеют тенденцию работать хуже на новых данных, чем на их данных обучения.
Переоснащение будет главной темой в главе 3.

На этом мы завершаем наш самый первый пример - вы только что увидели, как мы можем построить и обучить нейронную сеть для классификации рукописных цифр в менее 20 строк кода Python. В следующей главе мы подробно рассмотрим каждую движущуюся часть, которую мы только что просмотрели, и выясним, что на самом деле происходит за кулисами. Вы узнаете о «тензорах», объектах хранения данных, поступающих в сеть, о тензорных операциях, где слои созаны с помощью градиентного спуска, что позволяет нашей сети учиться на примерах обучения.